##How do they Visualize?
A basic project vizualizing the learning process of Neural Networks

Before moving forward make sure you activate the Hardware accelerator (GPU) on this notebook. For the basics, you get the T4 GPU for free. Use it

Step 1: Update certain Dependencies

In [ ]:
!pip install --upgrade tf-keras-vis tensorflow

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus

_, gpus = num_of_gpus()
print('{} GPUs'.format(gpus)) #check if gpu available. If its zero, use hardware accelerator.


1 GPUs


Step2: Import the VGG 16 model for testing.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16 as Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

# Load model
model = Model(weights='imagenet', include_top=True)
#model.summary()  #Uncomment this part to see the underlying structure of the model.

Step3: Download the test images

In [ ]:
#Load the example datasets
!wget https://github.com/keisen/tf-keras-vis/raw/master/docs/examples/images/goldfish.jpg
!wget https://github.com/keisen/tf-keras-vis/raw/master/docs/examples/images/bear.jpg
!wget https://github.com/keisen/tf-keras-vis/raw/master/docs/examples/images/soldiers.jpg

Step4: Load and Display the text images

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

# Image titles
image_titles = ['Goldfish', 'Bear', 'Assault rifle']

# Load images
img1 = load_img('goldfish.jpg', target_size=(224, 224))
img2 = load_img('bear.jpg', target_size=(224, 224))
img3 = load_img('soldiers.jpg', target_size=(224, 224))
images = np.asarray([np.array(img1), np.array(img2), np.array(img3)])

# Preparing input data
X = preprocess_input(images)

# Code to display the image
subplot_args = { 'nrows': 1, 'ncols': 3, 'figsize': (9, 3),
                 'subplot_kw': {'xticks': [], 'yticks': []} }
f, ax = plt.subplots(**subplot_args)
for i, title in enumerate(image_titles):
    ax[i].set_title(title, fontsize=14)
    ax[i].imshow(images[i])
plt.tight_layout()
plt.show()

Step5: Load the model outputs and modify it to get the raw outputs for the visuzalization.

In [ ]:
def loss(output):
    # 1 is the imagenet index corresponding to Goldfish, 294 to Bear and 413 to Assault Rifle.
    return (output[0][1], output[1][294], output[2][413])
def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

Step6: We run different vizualization models and see the output

Grad-CAM, or Gradient-weighted Class Activation Mapping, is a visualization technique for deep learning networks that helps in understanding which parts of a given image led a Convolutional Neural Network (CNN) to its final decision. It uses the gradients of any target concept, flowing into the final convolutional layer to produce a coarse localization map highlighting the important regions in the image for predicting the concept. This technique is widely used for making CNN-based models more transparent and explainable.

In [ ]:
%%time
from tensorflow.keras import backend as K
from tf_keras_vis.utils import normalize
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

# Create Gradcam object
gradcam = Gradcam(model,
                  model_modifier=model_modifier,
                  clone=False)

# Generate heatmap with GradCAM
cam = gradcam(loss,
              X,
              penultimate_layer=-1, # model.layers number
             )
cam = normalize(cam)

#below code is to display the heatmaps for the gradcam

f, ax = plt.subplots(**subplot_args)
for i, title in enumerate(image_titles):
    heatmap = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=14)
    ax[i].imshow(images[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.5) # overlay
plt.tight_layout()
plt.show()

Grad-CAM++ is an advanced version of Grad-CAM, offering improvements in the visualization of convolutional neural networks (CNNs). It extends the Grad-CAM method by using a weighted combination of the positive partial derivatives of the last convolutional layer’s feature maps with respect to a specific class score. This results in more accurate and detailed visual explanations, highlighting multiple relevant regions in the image instead of just one. Grad-CAM++ is particularly useful for identifying fine-grained features that contribute to a CNN’s decision, making it a valuable tool for interpreting complex models.

In [ ]:
#CHECKPOINT 1 : Implement the Gradcampluplus function, following through the same code as above and replace the gradcam function with gradcamplusplus and check the results

Score-CAM, or Score-weighted Class Activation Mapping, is an interpretability method for Convolutional Neural Networks (CNNs) that builds on the idea of Grad-CAM. Unlike Grad-CAM, which uses gradient information to weigh the activation maps, Score-CAM uses the scores of the target class before the softmax layer to create a weighted combination of activation maps. This approach avoids potential issues with using gradients and provides a more straightforward and effective visualization of the areas in an image that contribute to a model’s decision.

In [ ]:
%%time

from tf_keras_vis.scorecam import ScoreCAM

# Create ScoreCAM object
scorecam = ScoreCAM(model, model_modifier, clone=False)
if gpus > 0:
    # Generate heatmap with ScoreCAM
    cam = scorecam(loss,
                   X,
                   penultimate_layer=-1, # model.layers number
                  )
    cam = normalize(cam)

    f, ax = plt.subplots(**subplot_args)
    for i, title in enumerate(image_titles):
        heatmap = np.uint8(cm.jet(cam[i])[..., :3] * 255)
        ax[i].set_title(title, fontsize=14)
        ax[i].imshow(images[i])
        ax[i].imshow(heatmap, cmap='jet', alpha=0.5)
    plt.tight_layout()
    plt.show()
else:
    print("NOTE: Change to GPU to see visual output\n")

Faster Score-CAM appears to be a variant of Score-CAM that is designed to generate visual explanations more quickly. While the standard Score-CAM method avoids the use of gradients and instead uses forward passing scores to weigh activation maps for visual explanations, Faster Score-CAM likely incorporates optimizations that speed up this process. This could involve more efficient computation or approximation techniques to reduce the time required to visualize Class Activation Maps (CAMs) while still highlighting the influential regions in an image for a CNN’s decision-making process

In [ ]:
%%time

# Create ScoreCAM object
scorecam = ScoreCAM(model, model_modifier, clone=False)

# Generate heatmap with Faster-ScoreCAM
cam = scorecam(loss,
               X,
               penultimate_layer=-1, # model.layers number
               max_N=10
              )
cam = normalize(cam)

f, ax = plt.subplots(**subplot_args)
for i, title in enumerate(image_titles):
    heatmap = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=14)
    ax[i].imshow(images[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.5)
plt.tight_layout()
plt.show()